In [1]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00


In [4]:
import os
import re
import string
import json
import numpy as np
import pandas as pd
import random
from sklearn import metrics, model_selection
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

In [11]:
df = pd.read_csv("./CPTALGOML.csv")

In [12]:
df["qns"] = df["question_text"] + "\n" + df["input_outputs"]
df = df[["qns", "algo_tags"]]
df.head()

,qns,algo_tags
0,This is the hard version of the problem. The o...,['math']
1,This is the simple version of the problem. The...,"['greedy', 'math']"
2,You are given an array a consisting of n zeros...,"['brute force', 'search']"
3,Timofey has an apple tree growing in his garde...,['math']
4,Vanya really likes math. One day when he was s...,['math']


In [13]:
idxs = set(range(df.shape[0]))
train_idx = set(random.sample(idxs, k=int(df.shape[0]*0.9)))
len(train_idx)

5829

In [14]:
df_train = df.iloc[list(train_idx), :]
df_dev = df.iloc[list(idxs.difference(train_idx)), :]

df_train.shape, df_dev.shape

((5829, 2), (648, 2))

In [15]:
with open("./algo_classes.json") as f:
  algo_mapping = json.load(f)

algo_mapping

{'brute force': 0,
 'dynamic programming': 1,
 'greedy': 2,
 'math': 3,
 'search': 4}

In [16]:
for tag in algo_mapping:
  df_train[tag] = np.zeros(shape=(df_train.shape[0], 1))
  df_dev[tag] = np.zeros(shape=(df_dev.shape[0], 1), dtype=np.int16)

for i in range(df_train.shape[0]):
  tags = df_train.iloc[i, 1].split("'")
  for s in tags:
    if s in algo_mapping:
      df_train.iat[i, algo_mapping[s]+2] = 1

for i in range(df_dev.shape[0]):
  tags = df_dev.iloc[i, 1].split("'")
  for s in tags:
    if s in algo_mapping:
      df_dev.iat[i, algo_mapping[s]+2] = 1

In [17]:
df_train = df_train.reset_index(drop=True)
df_dev = df_dev.reset_index(drop=True)

In [18]:
df_train.shape, df_dev.shape

((5829, 7), (648, 7))

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [20]:
MAX_LEN = 500
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 10
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [21]:
target_cols = [col for col in df_train.columns if col not in ['qns', 'algo_tags']]
target_cols

['brute force', 'dynamic programming', 'greedy', 'math', 'search']

In [22]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.qns
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [23]:
train_dataset = BERTDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = BERTDataset(df_dev, tokenizer, MAX_LEN)

In [24]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE,
                          num_workers=4, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE,
                          num_workers=4, shuffle=False, pin_memory=True)

In [25]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,5)

    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

model = BERTClass()
model.to(device);

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [27]:
optimizer = AdamW(params =  model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

In [28]:
def train(epoch):
    model.train()
    losses = 0
    for _,data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, targets)
        losses += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch: {epoch}, loss: {losses/len(train_loader)}")


In [29]:
for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, loss: 0.4757691302294594
Epoch: 1, loss: 0.41949859138869455
Epoch: 2, loss: 0.383912908462667
Epoch: 3, loss: 0.34290280177760024
Epoch: 4, loss: 0.29670739663951357
Epoch: 5, loss: 0.24814252991265068
Epoch: 6, loss: 0.20595259711003272
Epoch: 7, loss: 0.1610853474955132
Epoch: 8, loss: 0.13362792879087526
Epoch: 9, loss: 0.1129743787325888


In [30]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [31]:
outputs, targets = validation()
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.3425925925925926
F1 Score (Micro) = 0.574944071588367
F1 Score (Macro) = 0.5328284966956871
